In [ ]:
import glob
import os
import subprocess
import sys

sys.path.append("./src")
try:
    sys.path.append(
        glob.glob(
            "%s/PythonAPI/carla/dist/carla-*%d.%d-%s.egg"
            % (
                os.environ.get("CARLA_ROOT", "."),
                sys.version_info.major,
                sys.version_info.minor,
                "win-amd64" if os.name == "nt" else "linux-x86_64",
            )
        )[0]
    )
except IndexError:
    pass

In [ ]:
import carla
from carla_world import CarlaWorld

### Initilize Connection

In [ ]:
# Define settings
sync = False
fixed_delta_seconds = None
reload = True
host = "127.0.0.1"
port = 2000
map_name = "Town01"
npc_vehicle_num = 60
npc_walker_num = 20

In [ ]:
client = carla.Client(host, port)
client.set_timeout(10.0)

In [ ]:
# Test if the server is ready
print(client.get_server_version())

### Get static assets

In [ ]:
carla_world = CarlaWorld(client)
if map_name not in carla_world.map.name:
    carla_world.load_world(map_name)
elif reload:
    carla_world.reload_world()

carla_world.toggle_sync_mode(sync, fixed_delta_seconds)

### Get Ego Actor

In [ ]:
# If you already have a server running with vehicles
# you can use the following code to get the vehicle you want
hero_actor = carla_world.find_actor_by_name("hero")
if hero_actor is None:
    print("No hero vehicle found, spawning a new one")
    hero_actor = carla_world.spawn_actor("hero", "vehicle.tesla.model3")

carla_world.tick()
carla_world.set_view(hero_actor.get_transform(), "behind")

In [ ]:
carla_world.follow_actor_view(hero_actor, "above")

In [ ]:
# carla_world.stop_follow_actor_view()

### Manual Control

In [ ]:
control_proc = subprocess.Popen(
    [
        "python",
        "examples/manual_control.py",
        "--host",
        host,
        "--port",
        str(port),
        "--rolename",
        hero_actor.attributes["role_name"],
        "--res",
        "800x600",
        "--keep_ego_vehicle",
    ],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
)

### Generate traffic

In [ ]:
traffic_proc = subprocess.Popen(
    [
        "python",
        "examples/generate_traffic.py",
        "--host",
        host,
        "--port",
        str(port),
        "-n",
        str(npc_vehicle_num),
        "-w",
        str(npc_walker_num),
        "--safe",
        "--asynch",
    ],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
)

### Render Bounding Box to Camera

In [ ]:
bbox_proc = subprocess.Popen(
    [
        "python",
        "examples/sensor_bounding_box.py",
        "--host",
        host,
        "--port",
        str(port),
        "--rolename",
        hero_actor.attributes["role_name"],
        "--view",
        "above",
    ],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT,
)

### Test Space, Do what you like

In [ ]:
from local_carla_api import *